<a href="https://colab.research.google.com/github/Gutobastos/imersao-iA-3-Alura/blob/main/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai
%pip -q install google-adk

In [3]:
import os

# E necessário inserir a chave na variável do seu ambiente com sua GOOGLE_API_KEY
google_api_key = os.environ.get('GOOGLE_API_KEY')
if google_api_key:
    print("Chave da API do Google encontrada nas variáveis de ambiente do seu PC.\n")
else:
    print("Chave da API do Google não encontrada nas variáveis de ambiente e tentaremos pelo GOOGLE COLAB.\n")
    #from google.colab import userdata
    #os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

Chave da API do Google encontrada nas variáveis de ambiente do seu PC.



In [4]:
from google import genai
client = genai.Client()
modelo = "gemini-2.0-flash"

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

In [ ]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name = "agente_buscador",
      model = "gemini-2.0-flash",
      description = "Agente que busca informações no Google",
      tools = [google_search],
      instruction = """
      Você é um assistente de pésquisa. E sua tarefa é usar a ferramenta do Google (google_search)
      para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
      Foque no máximo de 5 lançamentos relevantes, com base na qualidade e entusiasmo das notícias sobre ele.
      Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que não seja tão relevante assim
      e pode ser substituído por outro que tenha mais.
      Esses lançamentos relevantes devem ser atuais de no máximo um mês antes da data de hoje.
      """
  )
  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

  lancamentos = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejado de conteúdo, especialista em redes sociais.
        Com base nos lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano
        sobre quais são os pontos mais relevantes que poderíamos abordar em um
        post em cada um deles. Você também pode usar o (google_search) para
        encontrar mais informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base
        nas suas pesquisas e retornar esse tema, seus pontos mais relevantes e
        plano com os assuntos a serem abordados no post que será escrito
        posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado